<a href="https://colab.research.google.com/github/huggingface/deep-rl-class/blob/main/notebooks/unit6/unit6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unit 6: Advantage Actor Critic (A2C) using Robotics Simulations with Panda-Gym 🤖

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/thumbnail.png"  alt="Thumbnail"/>

In this notebook, you'll learn to use A2C with [Panda-Gym](https://github.com/qgallouedec/panda-gym). You're going **to train a robotic arm** (Franka Emika Panda robot) to perform a task:

- `Reach`: the robot must place its end-effector at a target position.

After that, you'll be able **to train in other robotics tasks**.


### 🎮 Environments:

- [Panda-Gym](https://github.com/qgallouedec/panda-gym)

###📚 RL-Library:

- [Stable-Baselines3](https://stable-baselines3.readthedocs.io/)

We're constantly trying to improve our tutorials, so **if you find some issues in this notebook**, please [open an issue on the GitHub Repo](https://github.com/huggingface/deep-rl-class/issues).

## Objectives of this notebook 🏆

At the end of the notebook, you will:

- Be able to use **Panda-Gym**, the environment library.
- Be able to **train robots using A2C**.
- Understand why **we need to normalize the input**.
- Be able to **push your trained agent and the code to the Hub** with a nice video replay and an evaluation score 🔥.




## This notebook is from the Deep Reinforcement Learning Course
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration"/>

In this free course, you will:

- 📖 Study Deep Reinforcement Learning in **theory and practice**.
- 🧑‍💻 Learn to **use famous Deep RL libraries** such as Stable Baselines3, RL Baselines3 Zoo, CleanRL and Sample Factory 2.0.
- 🤖 Train **agents in unique environments**

And more check 📚 the syllabus 👉 https://simoninithomas.github.io/deep-rl-course

Don’t forget to **<a href="http://eepurl.com/ic5ZUD">sign up to the course</a>** (we are collecting your email to be able to **send you the links when each Unit is published and give you information about the challenges and updates).**


The best way to keep in touch is to join our discord server to exchange with the community and with us 👉🏻 https://discord.gg/ydHrjt3WP5

## Prerequisites 🏗️
Before diving into the notebook, you need to:

🔲 📚 Study [Actor-Critic methods by reading Unit 6](https://huggingface.co/deep-rl-course/unit6/introduction) 🤗  

# Let's train our first robots 🤖

To validate this hands-on for the [certification process](https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process),  you need to push your trained model to the Hub and get the following results:

- `PandaReachDense-v3` get a result of >= -3.5.

To find your result, go to the [leaderboard](https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard) and find your model, **the result = mean_reward - std of reward**

For more information about the certification process, check this section 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process

## Set the GPU 💪
- To **accelerate the agent's training, we'll use a GPU**. To do that, go to `Runtime > Change Runtime type`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Step 1">

- `Hardware Accelerator > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2">

## Create a virtual display 🔽

During the notebook, we'll need to generate a replay video. To do so, with colab, **we need to have a virtual screen to be able to render the environment** (and thus record the frames).

Hence the following cell will install the librairies and create and run a virtual screen 🖥

In [14]:
%%capture
# !apt install python-opengl
# !apt install ffmpeg
# !apt install xvfb
# !pip3 install pyvirtualdisplay

In [15]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

### Install dependencies 🔽

The first step is to install the dependencies, we’ll install multiple ones:
- `gymnasium`
- `panda-gym`: Contains the robotics arm environments.
- `stable-baselines3`: The SB3 deep reinforcement learning library.
- `huggingface_sb3`: Additional code for Stable-baselines3 to load and upload models from the Hugging Face 🤗 Hub.
- `huggingface_hub`: Library allowing anyone to work with the Hub repositories.

⏲ The installation can **take 10 minutes**.

In [16]:
# !pip install stable-baselines3[extra]
# !pip install gymnasium

In [17]:
# !pip install huggingface_sb3
# !pip install huggingface_hub
# !pip install panda_gym

## Import the packages 📦

In [18]:
import os

import gymnasium as gym
import panda_gym

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

## PandaReachDense-v3 🦾

The agent we're going to train is a robotic arm that needs to do controls (moving the arm and using the end-effector).

In robotics, the *end-effector* is the device at the end of a robotic arm designed to interact with the environment.

In `PandaReach`, the robot must place its end-effector at a target position (green ball).

We're going to use the dense version of this environment. It means we'll get a *dense reward function* that **will provide a reward at each timestep** (the closer the agent is to completing the task, the higher the reward). Contrary to a *sparse reward function* where the environment **return a reward if and only if the task is completed**.

Also, we're going to use the *End-effector displacement control*, it means the **action corresponds to the displacement of the end-effector**. We don't control the individual motion of each joint (joint control).

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/robotics.jpg"  alt="Robotics"/>


This way **the training will be easier**.



### Create the environment

#### The environment 🎮

In `PandaReachDense-v3` the robotic arm must place its end-effector at a target position (green ball).

In [19]:
env_id = "PandaReachDense-v3"

# Create the env
env = gym.make(env_id)

# Get the state space and action space
s_size = env.observation_space.shape
a_size = env.action_space

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


In [20]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Get a random observation

_____OBSERVATION SPACE_____ 

The State Space is:  None
Sample observation {'achieved_goal': array([-2.1884394,  8.445852 ,  4.128144 ], dtype=float32), 'desired_goal': array([ 4.1493225,  8.870105 , -7.571747 ], dtype=float32), 'observation': array([-0.72712356,  2.5578456 , -4.8560247 , -9.348572  , -8.101642  ,
       -4.2701235 ], dtype=float32)}


The observation space **is a dictionary with 3 different elements**:
- `achieved_goal`: (x,y,z) the current position of the end-effector.
- `desired_goal`: (x,y,z) the target position for the end-effector.
- `observation`: position (x,y,z) and velocity of the end-effector (vx, vy, vz).

Given it's a dictionary as observation, **we will need to use a MultiInputPolicy policy instead of MlpPolicy**.

In [21]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

The Action Space is:  Box(-1.0, 1.0, (3,), float32)
Action Space Sample [ 0.8337736   0.623979   -0.20966296]


The action space is a vector with 3 values:
- Control x, y, z movement

### Normalize observation and rewards

A good practice in reinforcement learning is to [normalize input features](https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html).

For that purpose, there is a wrapper that will compute a running average and standard deviation of input features.

We also normalize rewards with this same wrapper by adding `norm_reward = True`

[You should check the documentation to fill this cell](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecnormalize)

In [22]:
# env = make_vec_env(env_id, n_envs=4)

# # Adding this wrapper to normalize the observation and the reward
# env = # TODO: Add the wrapper

#### Solution

In [23]:
env = make_vec_env(env_id, n_envs=4)

env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


### Create the A2C Model 🤖

For more information about A2C implementation with StableBaselines3 check: https://stable-baselines3.readthedocs.io/en/master/modules/a2c.html#notes

To find the best parameters I checked the [official trained agents by Stable-Baselines3 team](https://huggingface.co/sb3).

In [24]:
# model = # Create the A2C model and try to find the best parameters

#### Solution

In [25]:
model = A2C(policy = "MultiInputPolicy",
            env = env,
            verbose=1)

Using cuda device


### Train the A2C agent 🏃
- Let's train our agent for 1,000,000 timesteps, don't forget to use GPU on Colab. It will take approximately ~25-40min

In [26]:
model.learn(1_000_000)

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 43.2     |
|    ep_rew_mean        | -10.3    |
|    success_rate       | 0.222    |
| time/                 |          |
|    fps                | 634      |
|    iterations         | 100      |
|    time_elapsed       | 3        |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -4.22    |
|    explained_variance | 0.914    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -0.729   |
|    std                | 0.988    |
|    value_loss         | 0.0749   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 45       |
|    ep_rew_mean        | -10.4    |
|    success_rate       | 0.172    |
| time/                 |          |
|    fps                | 639      |
|    iterations         | 200      |
|

In [27]:
# Save the model and  VecNormalize statistics when saving the agent
model.save("a2c-PandaReachDense-v3")
env.save("vec_normalize.pkl")

### Evaluate the agent 📈
- Now that's our  agent is trained, we need to **check its performance**.
- Stable-Baselines3 provides a method to do that: `evaluate_policy`

In [28]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Load the saved statistics
eval_env = DummyVecEnv([lambda: gym.make("PandaReachDense-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

# We need to override the render_mode
eval_env.render_mode = "rgb_array"
 
#  do not update them at test time
eval_env.training = False
# reward normalization is not needed at test time
eval_env.norm_reward = False

# Load the agent
model = A2C.load("a2c-PandaReachDense-v3")

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
Mean reward = -0.22 +/- 0.10


/home/libing/.conda/envs/deep-rl-class/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:71: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


### Publish your trained model on the Hub 🔥
Now that we saw we got good results after the training, we can publish our trained model on the Hub with one line of code.

📚 The libraries documentation 👉 https://github.com/huggingface/huggingface_sb3/tree/main#hugging-face--x-stable-baselines3-v20


By using `package_to_hub`, as we already mentionned in the former units, **you evaluate, record a replay, generate a model card of your agent and push it to the hub**.

This way:
- You can **showcase our work** 🔥
- You can **visualize your agent playing** 👀
- You can **share with the community an agent that others can use** 💾
- You can **access a leaderboard 🏆 to see how well your agent is performing compared to your classmates** 👉 https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard


To be able to share your model with the community there are three more steps to follow:

1️⃣ (If it's not already done) create an account to HF ➡ https://huggingface.co/join

2️⃣ Sign in and then, you need to store your authentication token from the Hugging Face website.
- Create a new token (https://huggingface.co/settings/tokens) **with write role**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="Create HF Token">

- Copy the token
- Run the cell below and paste the token

In [29]:
notebook_login()
!git config --global credential.helper store

In [ ]:
# evaluate and save the model

from pathlib import Path
import datetime
import json
import os

import numpy as np

# Video recording (SB3-friendly)

def record_video_sb3(env_id, model, out_path, video_length=1000, fps=30, deterministic=True):
  """Record one episode (or up to `video_length` steps) for a Stable-Baselines3 model.

  This works for SB3 models (A2C/PPO/SAC/...) that implement `predict()`.
  It uses Gymnasium's RecordVideo wrapper so we don't keep all frames in RAM.
  """
  try:
    import gymnasium as gym
    from gymnasium.wrappers import RecordVideo
  except Exception:
    import gym
    from gym.wrappers import RecordVideo

  out_path = Path(out_path)
  out_path.parent.mkdir(parents=True, exist_ok=True)

  env = gym.make(env_id, render_mode="rgb_array")
  # some wrappers look at this metadata
  try:
    env.metadata["render_fps"] = fps
  except Exception:
    pass

  # Record the first episode
  env = RecordVideo(
      env,
      video_folder=str(out_path.parent),
      name_prefix=out_path.stem,
      episode_trigger=lambda ep: ep == 0,
  )

  obs = env.reset()
  if isinstance(obs, tuple):
    obs = obs[0]

  for _ in range(int(video_length)):
    action, _ = model.predict(obs, deterministic=deterministic)
    step_out = env.step(action)
    if len(step_out) == 5:
      obs, reward, terminated, truncated, info = step_out
      done = terminated or truncated
    else:
      obs, reward, done, info = step_out

    if done:
      break

  env.close()

  # Gymnasium will create something like <prefix>-episode-0.mp4
  # Try to rename it to exactly out_path for convenience.
  try:
    created = sorted(out_path.parent.glob(out_path.stem + "*.mp4"))
    if created:
      created[0].replace(out_path)
  except Exception:
    pass


In [ ]:
def save_and_evaluate(
                model,
                env_id,
                eval_env,
                video_fps=30,
                n_eval_episodes=10,
                video_length=1000,
                deterministic=True,
                ):
  """Save + evaluate locally for Stable-Baselines3 models.

  Fixes the previous video failure: SB3 models use `model.predict()`, not `policy.act()`.
  """

  from stable_baselines3.common.evaluation import evaluate_policy

  def _json_default(o):
    import numpy as np
    if isinstance(o, (np.integer,)):
      return int(o)
    if isinstance(o, (np.floating,)):
      return float(o)
    if isinstance(o, (np.ndarray,)):
      return o.tolist()
    raise TypeError(f"Object of type {type(o).__name__} is not JSON serializable")
  run_id = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  out_dir = Path("sb3_exports") / str(env_id) / run_id
  out_dir.mkdir(parents=True, exist_ok=True)

  # 1) Save model (SB3 way)
  model_path = out_dir / "model"
  model.save(str(model_path))

  # 2) Evaluate
  mean_reward, std_reward = evaluate_policy(
      model,
      eval_env,
      n_eval_episodes=int(n_eval_episodes),
      deterministic=deterministic,
  )

  # 3) Write results
  results = {
      "env_id": str(env_id),
      "mean_reward": float(mean_reward),
      "std_reward": float(std_reward),
      "n_eval_episodes": int(n_eval_episodes),
      "eval_datetime": datetime.datetime.now().isoformat(),
  }
  with open(out_dir / "results.json", "w") as outfile:
    json.dump(results, outfile, indent=2, default=_json_default)

  # 4) Record video (best effort)
  try:
    record_video_sb3(
        env_id=str(env_id),
        model=model,
        out_path=out_dir / "replay.mp4",
        video_length=video_length,
        fps=video_fps,
        deterministic=deterministic,
    )
  except Exception as e:
    print("[warn] video recording skipped:", repr(e))

  print(f"Saved to: {out_dir}")
  print(f"Eval mean_reward={mean_reward:.2f} std_reward={std_reward:.2f}")

  return out_dir, mean_reward, std_reward


In [49]:

save_and_evaluate(model, # The model we want to save
                env_id,
                eval_env, # Evaluation environment
                video_fps=30
                )

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


/home/libing/.conda/envs/deep-rl-class/lib/python3.10/site-packages/gymnasium/wrappers/rendering.py:293: UserWarning: WARN: Overwriting existing videos at /home/libing/source/ml/rl/deep-rl-class/notebooks/unit6/sb3_exports/PandaPickAndPlace-v3/20260118-132712 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Saved to: sb3_exports/PandaPickAndPlace-v3/20260118-132712
Eval mean_reward=-50.00 std_reward=0.00


(PosixPath('sb3_exports/PandaPickAndPlace-v3/20260118-132712'),
 np.float64(-50.0),
 np.float64(0.0))

If you don't want to use a Google Colab or a Jupyter Notebook, you need to use this command instead: `huggingface-cli login`

3️⃣ We're now ready to push our trained agent to the 🤗 Hub 🔥 using `package_to_hub()` function

For this environment, **running this cell can take approximately 10min**

In [50]:
# from huggingface_sb3 import package_to_hub

# package_to_hub(
#     model=model,
#     model_name=f"a2c-{env_id}",
#     model_architecture="A2C",
#     env_id=env_id,
#     eval_env=eval_env,
#     repo_id=f"ThomasSimonini/a2c-{env_id}", # Change the username
#     commit_message="Initial commit",
# )

## Some additional challenges 🏆
The best way to learn **is to try things by your own**! Why not trying  `PandaPickAndPlace-v3`?

If you want to try more advanced tasks for panda-gym, you need to check what was done using **TQC or SAC** (a more sample-efficient algorithm suited for robotics tasks). In real robotics, you'll use a more sample-efficient algorithm for a simple reason: contrary to a simulation **if you move your robotic arm too much, you have a risk of breaking it**.

PandaPickAndPlace-v1 (this model uses the v1 version of the environment): https://huggingface.co/sb3/tqc-PandaPickAndPlace-v1

And don't hesitate to check panda-gym documentation here: https://panda-gym.readthedocs.io/en/latest/usage/train_with_sb3.html

We provide you the steps to train another agent (optional):

1. Define the environment called "PandaPickAndPlace-v3"
2. Make a vectorized environment
3. Add a wrapper to normalize the observations and rewards. [Check the documentation](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecnormalize)
4. Create the A2C Model (don't forget verbose=1 to print the training logs).
5. Train it for 1M Timesteps
6. Save the model and  VecNormalize statistics when saving the agent
7. Evaluate your agent
8. Publish your trained model on the Hub 🔥 with `package_to_hub`


### Solution (optional)

In [51]:
# 1 - 2
env_id = "PandaPickAndPlace-v3"
env = make_vec_env(env_id, n_envs=4)

# 3
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

# 4
model = A2C(policy = "MultiInputPolicy",
            env = env,
            verbose=1)
# 5
model.learn(1_000_000)

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
Using cuda device
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 48.8     |
|    ep_rew_mean        | -48.8    |
|    success_rate       | 0.025    |
| time/                 |          |
|    fps                | 701      |
|    iterations         | 100      |
|    time_elapsed       | 2        |
|    total_timesteps    | 2000   

In [52]:
# 6
model_name = "a2c-PandaPickAndPlace-v3";
model.save(model_name)
env.save("vec_normalize.pkl")

# 7
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Load the saved statistics
eval_env = DummyVecEnv([lambda: gym.make("PandaPickAndPlace-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

#  do not update them at test time
eval_env.training = False
# reward normalization is not needed at test time
eval_env.norm_reward = False

# Load the agent
model = A2C.load(model_name)

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

# 8
package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"ThomasSimonini/a2c-{env_id}", # TODO: Change the username
    commit_message="Initial commit",
)

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
Mean reward = -46.80 +/- 9.60
ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.


HfHubHTTPError: (Request ID: Root=1-696c74f8-600b4a8168aa64f12844ffdb;a4d53884-5177-49b6-a6d7-df03ffbdf20c)

403 Forbidden: You don't have the rights to create a model under the namespace "ThomasSimonini".
Cannot access content at: https://hf-mirror.com/api/repos/create.
Make sure your token has the correct permissions.

See you on Unit 7! 🔥
## Keep learning, stay awesome 🤗